In [1]:
import numpy as np
import pywt
from scipy.io import wavfile
import matplotlib.pyplot as plt
from collections import Counter
import huffman
from sklearn.tree import DecisionTreeRegressor


In [4]:
sampling_rate, data = wavfile.read('harvard.wav')


C:\Users\zalikapope\AppData\Local\Temp\ipykernel_1844\192127685.py:1: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sampling_rate, data = wavfile.read('harvard.wav')


In [5]:
coeffs = pywt.wavedec(data, 'db6')


In [6]:
regressors = []
thresholds = []

for i, coeff in enumerate(coeffs):
    regressor = DecisionTreeRegressor()
    flatten_coeff = coeff.flatten()
    x_train = np.arange(len(flatten_coeff)).reshape(-1, 1)
    y_train = flatten_coeff
    regressor.fit(x_train, y_train)
    regressors.append(regressor)
    thresholds.append(np.mean(np.abs(regressor.predict(x_train) - y_train)))


In [7]:
compressed_coeffs = []
for i, coeff in enumerate(coeffs):
    compressed_coeff = pywt.threshold(coeff, thresholds[i], mode='soft')
    compressed_coeffs.append(compressed_coeff)


C:\Users\zalikapope\AppData\Local\Programs\Python\Python39\lib\site-packages\pywt\_thresholding.py:23: RuntimeWarning: invalid value encountered in divide
  thresholded = (1 - value/magnitude)


In [8]:
flatten_coeffs = np.hstack(compressed_coeffs)


In [9]:
tuple_coeffs = [tuple(coeff) for coeff in flatten_coeffs]
symbol_counts = dict(Counter(tuple_coeffs))
huff_tree = huffman.codebook(symbol_counts)
huffman_dict = dict(huff_tree)


In [13]:
encoded_coeffs = [huffman_dict.get(symbol, '') for symbol in tuple_coeffs]
binary_data = ''.join(encoded_coeffs)

In [14]:
padding_length = 8 - len(binary_data) % 8
binary_data += "0" * padding_length


In [15]:
byte_array = bytearray()
for i in range(0, len(binary_data), 8):
    byte = binary_data[i:i + 8]
    byte_array.append(int(byte, 2))

In [16]:
flattened_coeffs = np.concatenate(compressed_coeffs)
wavfile.write('compressed.wav', sampling_rate, flattened_coeffs)
np.savez('compressed.npz', huffman_dict=huffman_dict, compressed_coeffs=compressed_coeffs)


In [17]:
print("\nCompressed Audio Data:")
print(flattened_coeffs)

print("\nHuffman Dictionary:")
#print(huffman_dict)

print("\nHuffman Encoding Statistics:")
#print(symbol_counts)

print("\nNumber of Bits before Compression:", len(data) * 16)  # Assuming 16 bits per sample in the original audio
print("Number of Bits after Compression:", len(byte_array) * 8)
compression_ratio = (len(byte_array) * 8) / (len(data) * 16)
print("Compression Ratio:", compression_ratio)



Compressed Audio Data:
[[ nan  nan]
 [ nan   1.]
 [ -1.  -1.]
 ...
 [-57. -57.]
 [-57. -58.]
 [-61. -60.]]

Huffman Dictionary:

Huffman Encoding Statistics:

Number of Bits before Compression: 12952128
Number of Bits after Compression: 8
Compression Ratio: 6.176591213428404e-07
